In [ ]:
import bloqade
from bloqade import squin
import numpy as np
import stim
import tsim
import time

In [ ]:
@squin.kernel
def main():
    q = squin.qalloc(3)
    squin.broadcast.reset(q)
    squin.h(q[0])
    squin.t(q[0])
    squin.h(q[0])
    squin.cx(q[0], q[2])
    for i in [0, 2]:
        squin.depolarize(0.5, q[i])
        squin.cx(q[i], q[1])
    squin.measure(q[1])
    squin.cx(q[1], q[0])
    squin.broadcast.measure([q[0], q[2]])

c = bloqade.tsim.Circuit(main)
c.diagram(height=150)

In [ ]:
sampler = c.compile_sampler()

In [ ]:
samples = sampler.sample(shots=10_000)
assert np.all(samples[:, 1] == samples[:, 2])
samples

In [ ]:
c.diagram("pyzx", scale=35);

Here, TSIM makes use of _doubled_ notation:


<img src="../figures/doubling.png" alt="doubled notation" width=450px>

In [ ]:
tsim.Circuit("M 0").diagram("pyzx");

In [ ]:
tsim.Circuit("X 0 \nR 0").diagram("pyzx");

In [ ]:
tsim.Circuit("""
M !0
CNOT rec[-1] 1
""").diagram("pyzx")

In [ ]:
c = tsim.Circuit("""
RX 0
R 1 2 3 4 5
T 0
H 0
CNOT 0 2
CNOT 0 3
CNOT 2 5
TICK
CX 0 1 3 4
X_ERROR(0.01) 0 1 3 4
TICK
CX 2 1 5 4
X_ERROR(0.01) 2 1 5 4
TICK
MR 1 4
DETECTOR rec[-2]
DETECTOR rec[-1]
TICK
CX 0 1 3 4
X_ERROR(0.01) 0 1 3 4
TICK
CX 2 1 5 4
X_ERROR(0.01) 2 1 5 4
TICK
MR 1 4
DETECTOR rec[-2] rec[-4]
DETECTOR rec[-1] rec[-3]
M 0 2 3 5
DETECTOR rec[-3] rec[-4] rec[-6]
DETECTOR rec[-1] rec[-2] rec[-5]
OBSERVABLE_INCLUDE(0) rec[-3]
OBSERVABLE_INCLUDE(1) rec[-1]
""")

In [ ]:
c.diagram(height=200)

In [ ]:
c.diagram("pyzx", scale=30, scale_horizontally=1.4)

<img src="../figures/xor.png" alt="XOR gate" width=350px>

In [ ]:
c.diagram("pyzx-dets");

# Clifford sampling

In [ ]:
p = 0.0001
stim_circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_z",
    rounds=7,
    distance=7,
    before_round_data_depolarization=p,
    before_measure_flip_probability=p,
    after_clifford_depolarization=p,
    after_reset_flip_probability=p,
)
circuit = tsim.Circuit.from_stim_program(stim_circuit)
sampler = circuit.compile_detector_sampler()

stim_sampler = stim_circuit.compile_detector_sampler()

In [ ]:
circuit.without_noise().diagram("timeslice-svg", height=250, rows=1)

In [ ]:
# circuit.diagram("pyzx", scale_horizontally=2)

In [ ]:
num_samples = 100_000

start = time.perf_counter()
samples = sampler.sample(shots=num_samples)
end = time.perf_counter()
print(f"Time per shot (tsim): {(end - start) * 1e6 / num_samples:.2f} us")

In [ ]:
num_samples = 100_000

start = time.perf_counter()
samples = stim_sampler.sample(shots=num_samples)
end = time.perf_counter()
print(f"Time per shot (stim): {(end - start) * 1e6 / num_samples:.2f} us")

In [ ]:
import sinter
import matplotlib.pyplot as plt

noise_vals = np.logspace(-2.5, -1.5, 4)
tasks = [
    sinter.Task(
        circuit=tsim.Circuit.from_stim_program(
            stim.Circuit.generated(
                "surface_code:rotated_memory_z",
                distance=distance,
                rounds=3,
                after_clifford_depolarization=noise,
            )
        ).cast_to_stim(),
        json_metadata={"p": noise, "distance": distance, "rounds": 3},
    )
    for noise in noise_vals
    for distance in [3, 5, 7]
]

collected_stats = sinter.collect(
    num_workers=2,
    tasks=tasks,
    decoders=["pymatching"],
    max_shots=1024 * 256,
    max_errors=1024 * 256,
    start_batch_size=1024 * 256,
    max_batch_size=1024 * 256,
)

fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata["p"],
    group_func=lambda stats: stats.json_metadata["distance"],
    failure_units_per_shot_func=lambda stats: stats.json_metadata["rounds"],
)
plt.plot(noise_vals, noise_vals, color="k", linestyle="--", lw=0.5, label="uncoded")
ax.loglog()
ax.set_xlabel("Physical Error Rate")
ax.set_ylabel(f"Probability of logical $|\\bar{1}\\rangle$")
ax.legend();

## Non-Clifford circuits

In [ ]:
c = tsim.Circuit.from_file("../circuits/msd_5.stim")
c.diagram(height=400)

In [ ]:
sampler = c.compile_detector_sampler()

In [ ]:
num_samples = 100_000

start = time.perf_counter()
samples = sampler.sample(shots=num_samples)
end = time.perf_counter()
print(f"Time per shot (tsim): {(end - start) * 1e6 / num_samples:.2f} us")